# Mount Azure Storage

This notebook mounts the `dataservicesdlsdev` storage account to Databricks.

**Run this notebook ONCE before running the bronze ingestion notebook.**

## What it does:
- Mounts storage account at `/mnt/dataservicesdlsdev`
- Provides access to `mg-gold-raw-files` container
- Verifies mount was successful

## Required:
- Storage account access key (get from Azure Portal)
- OR Azure Key Vault with secrets configured

In [0]:
# MOUNT AZURE STORAGE - dataservicesdlsdev
# This mounts the entire storage account

print("=" * 80)
print("MOUNTING AZURE STORAGE")
print("=" * 80)

storage_account = "dataservicesdlsdev"
mount_point = f"/mnt/{storage_account}"

print(f"\nStorage Account: {storage_account}")
print(f"Mount Point: {mount_point}")

# Check if already mounted
already_mounted = False
try:
    dbutils.fs.ls(mount_point)
    print(f"\n✓ Storage already mounted at {mount_point}")
    already_mounted = True
except:
    print(f"\nMount does not exist. Creating mount...")

In [0]:
# CONFIGURE AUTHENTICATION
# Choose ONE option below:

# OPTION A: Hardcoded Access Key (QUICK START - FOR TESTING ONLY)
# Get your key from: Azure Portal > Storage Account > Access Keys > Show > Copy key1
# SECURITY WARNING: Replace with Key Vault (Option B) before production

access_key = "YOUR_ACCESS_KEY_HERE"  # REPLACE THIS WITH YOUR ACTUAL KEY

# OPTION B: From Azure Key Vault (RECOMMENDED)
# Uncomment these lines and comment out Option A:
# access_key = dbutils.secrets.get("bronze-ingestion", "adls-access-key")
# print("Using access key from Key Vault")

configs = {
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net": access_key
}

print("Authentication configured")
print(f"Access key length: {len(access_key)} characters (should be 88)")

In [0]:
# PERFORM MOUNT
if not already_mounted:
    try:
        dbutils.fs.mount(
            source=f"abfss://{storage_account}.dfs.core.windows.net/",
            mount_point=mount_point,
            extra_configs=configs
        )
        print(f"\n✓ Successfully mounted {mount_point}")
    except Exception as e:
        print(f"\n✗ Error mounting: {e}")
        print("\nTroubleshooting:")
        print("1. Verify access key is correct (88 characters)")
        print("2. Check storage account name: dataservicesdlsdev")
        print("3. Ensure storage account exists in Azure")
        raise
else:
    print("\nSkipping mount (already exists)")

In [0]:
# VERIFY MOUNT - LIST CONTAINERS
print("\n" + "=" * 80)
print("VERIFYING MOUNT - LISTING CONTAINERS")
print("=" * 80)

try:
    containers = dbutils.fs.ls(mount_point)
    print(f"\nFound {len(containers)} containers:")
    for container in containers:
        print(f"  - {container.name}")
except Exception as e:
    print(f"Error: {e}")

In [0]:
# CHECK MG-GOLD-RAW-FILES CONTAINER
print("\n" + "=" * 80)
print("CHECKING mg-gold-raw-files CONTAINER")
print("=" * 80)

mg_gold_path = f"{mount_point}/mg-gold-raw-files"

try:
    files = dbutils.fs.ls(mg_gold_path)
    print(f"\n✓ Container accessible: {mg_gold_path}")
    print(f"✓ Found {len(files)} items")
    
    if len(files) > 0:
        print("\nContents:")
        display(dbutils.fs.ls(mg_gold_path))
    else:
        print("\n⚠ Container is empty")
        print("Upload MercuryGate export files to begin processing")
        
except Exception as e:
    print(f"\n✗ Could not access mg-gold-raw-files: {e}")
    print("\nMake sure the container exists in Azure Portal")

In [0]:
# SUMMARY
print("\n" + "=" * 80)
print("MOUNT SETUP COMPLETE")
print("=" * 80)
print(f"\nMount Point: {mount_point}")
print(f"MG Files Path: {mount_point}/mg-gold-raw-files/")
print("\nNext Steps:")
print("1. Upload test files to mg-gold-raw-files container")
print("2. Run the bronze ingestion notebook")
print("3. Select domain and process files")
print("\n" + "=" * 80)